# Проверка качества данных: Источник ЕЦП.CRM

**Таблица:** `sandbox_ai.tmp_crm_svy`  
**Содержание:** мастер-система ФП/СФП — централизованное хранение факторов проблемности

## Загрузка данных

In [ ]:
import pandas as pd
import numpy as np

FILE_PATH = "../sources/data_crm.csv"
ENCODING = "utf-8-sig"
SEP = ";"

FIELD_DESCRIPTIONS = {
    "ROW_ID": "Уникальный ID фактора проблемности в системе CRM",
    "NUMBER_FP_SFP": "Номер ФП/СФП (например: ФП-558899)",
    "VAL": "Система-источник, откуда пришла информация о ФП/СФП",
    "VAL_1": "Статус ФП/СФП (Открыт, Закрыт и др.)",
    "TYPE_FP": "Тип: ФП (фактор проблемности) или СФП (существенный фактор проблемности)",
    "AGREEMENT_NUM": "Номер кредитного договора, к которому привязан ФП",
    "COMMENT_TEXT": "Результат рассмотрения на УОБ",
    "DATE_END_FP_SFP": "Дата снятия ФП/СФП с контроля",
    "DECISION_UOB": "Основания для закрытия ФП/СФП",
    "DEFOLT": "Признак дефолта клиента (Y/N)",
    "DESC_TEXT": "Результат урегулирования (комментарий)",
    "DESC_TEXT_1": "Результат урегулирования по сценарию (комментарий)",
    "END_DATE_SCR_FCT": "Фактическая дата окончания сценария",
    "END_DATE_SCR_PLAN": "Плановая дата окончания сценария",
    "END_EVENT_DATE_FACT": "Фактическая дата окончания мероприятия",
    "FACTOR_COMMENT": "Комментарий по сути фактора",
    "FIRST_END_DATE_EVENT": "Первоначальная плановая дата окончания мероприятия",
    "IDENTIFICATION_DATE": "Дата выявления ФП/СФП",
    "MORTGAGE_NUM": "Номер договора залога",
    "NEW_PLAN_END_DATE_EVT": "Новая плановая дата окончания мероприятия (при переносе)",
    "SCRIPT": "Сценарий урегулирования",
    "SUSPENSION_ISSUE": "Признак приостановления выдачи (Y/N)",
    "APPROVED": "Признак подтверждения ФП (Y/N)",
    "DISABLED": "Признак отклонения ФП (Y/N)",
    "ROW_ID_1": "ID сделки в CRM",
    "AGREEMENT_NUM_1": "Номер договора (из карточки сделки)",
    "AGREEMENT_OPEN_DT": "Дата открытия договора",
    "AGREEMENT_CLOSE_DT": "Дата закрытия договора",
    "AGR_OPEN_DT_FLG": "Признак договора с открытой датой (Y/N)",
    "APPROVED_SUM": "Сумма по договору",
    "CURCY_CD": "Валюта договора (RUB, USD, EUR, CNY)",
    "NAME": "Банковский продукт по договору",
    "VAL_2": "Стадия сделки (Действующая, Закрытая, Просроченная и др.)",
    "ROW_ID_2": "ID организации в CRM",
    "X_INN": "ИНН клиента (10 цифр — юрлицо, 12 — ИП)",
    "X_KPP": "КПП клиента (9 цифр)",
    "X_OGRN": "ОГРН (13 цифр — юрлицо, 15 — ИП)",
    "NAME_1": "Наименование организации",
    "SIC": "Фактический ОКВЭД",
    "STATUS": "Статус компании из внешнего источника (СПАРК)",
    "VAL_3": "Статус компании из внутреннего источника банка",
}

ID_COLUMNS = ["ROW_ID", "ROW_ID_1", "ROW_ID_2"]

In [ ]:
df = pd.read_csv(FILE_PATH, sep=SEP, encoding=ENCODING, dtype=str, low_memory=False)
df.columns = df.columns.str.strip()

print(f"Строк: {len(df):,}")
print(f"Колонок: {len(df.columns)}")
df.head()

## 1. Список всех полей с описанием

In [ ]:
fields_data = []
for i, col in enumerate(df.columns, start=1):
    desc = FIELD_DESCRIPTIONS.get(col, "(описание отсутствует)")
    fields_data.append({"\u2116": i, "Поле": col, "Описание": desc})

df_fields = pd.DataFrame(fields_data)
df_fields.style.hide(axis="index")

## 2. Пропуски и пустые значения

In [ ]:
total = len(df)

nulls_data = []
for col in df.columns:
    s = df[col]
    cnt_nan = s.isna().sum()
    s_str = s.dropna().astype(str).str.strip().str.lower()
    cnt_none = (s_str == "none").sum()
    cnt_nat = (s_str == "nat").sum()
    cnt_empty = (s_str == "").sum()
    cnt_total = cnt_nan + cnt_none + cnt_nat + cnt_empty
    pct = cnt_total / total * 100

    nulls_data.append({
        "Поле": col,
        "NaN": cnt_nan,
        "None": cnt_none,
        "NaT": cnt_nat,
        "Пустые": cnt_empty,
        "Итого": cnt_total,
        "%": round(pct, 1),
    })

df_nulls = pd.DataFrame(nulls_data)

df_nulls.style.hide(axis="index")

## 3. Полные дубликаты

In [ ]:
dup_mask = df.duplicated(keep=False)
n_dup_rows = dup_mask.sum()
n_extra_copies = df.duplicated(keep="first").sum()

print(f"Всего строк в таблице: {len(df):,}")
print(f"Строк, участвующих в дублировании: {n_dup_rows}")
print(f"Из них лишних копий: {n_extra_copies}")

if n_dup_rows > 0:
    dup_df = df[dup_mask].copy()
    dup_df["_group"] = dup_df.groupby(list(df.columns)).ngroup()
    dup_df["_count"] = dup_df.groupby("_group")["_group"].transform("count")
    dup_display = (
        dup_df.drop_duplicates(subset="_group")
        .sort_values("_group")
        .rename(columns={"_count": "Повторений"})
        .drop(columns="_group")
        .reset_index(drop=True)
    )
    print(f"\nДублирующиеся строки:")
    display(dup_display)
else:
    print("\nПолных дубликатов не обнаружено.")

## 4. Уникальность идентификаторов

In [ ]:
for id_col in ID_COLUMNS:
    if id_col not in df.columns:
        continue

    clean = df[id_col].dropna().astype(str).str.strip()
    clean = clean[clean != ""]

    n_total = len(clean)
    n_unique = clean.nunique()
    n_dupes = clean.duplicated().sum()

    print(f"--- {id_col} ---")
    print(f"  Заполнено: {n_total:,}")
    print(f"  Уникальных: {n_unique:,}")
    print(f"  Повторяющихся: {n_dupes}")

    if n_dupes > 0:
        dup_ids = clean[clean.duplicated(keep=False)]
        dup_detail = dup_ids.value_counts().reset_index()
        dup_detail.columns = [id_col, "Кол-во"]
        print(f"\n  Повторяющиеся значения:")
        display(dup_detail)
    print()